In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("camera.csv")
data

,index,name,price,number_of_comment,brand,size,weight(g),camera(mp),comment
0,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,Bạn ơi mình đã nhận được hàng rồi cám ơn Tiki ...
1,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,Tiki giao hàng nhanh giao đúng sản phẩm Máy đẹ...
2,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,Máy mình in ảnh tấm thì tối thui tấm thì trắng...
3,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,Máy cực kỳ xinh Chụp tốt
4,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,Bóng đèn flash có bụi đen và xuất hiện trên ảnh
...,...,...,...,...,...,...,...,...,...
1584,196,Máy ảnh Olympus OM-D E-M10 Mark III + Kit 14-4...,14500000,0,Olympus,NaN,NaN,NaN,NaN
1585,197,Máy Ảnh Fujifilm X-Pro 2 + 23mm F2 Black - Hàn...,50480000,0,Fujifilm,Dài 1405mm x Rộng 828mm x Cao 459mm,NaN,NaN,NaN
1586,198,Máy Ảnh Sony DSC-RX100M5A - Hàng Chính Hãng,20990000,0,Sony,1016 x 581 x 410 mm,NaN,NaN,NaN
1587,199,Máy ảnh Olympus E-M1X (Body) ( Hàng Chính hãng),72000000,0,Olympus,NaN,NaN,NaN,NaN


In [3]:
data['number_of_comment'].isin([0]).sum()

120

In [4]:
review_predict = data['comment']

In [5]:
sentiment = pd.read_csv('text.csv',index_col=[0])
sentiment

,text_lb,text
2,1.0,"Ấn tượng, tôi sẽ quay lại vào dịp nghỉ lần sau"
3,1.0,"Quá tuyệt vời , view đẹp , chủ nhà thân thiện ..."
6,1.0,"Phòng nghỉ đáng giá, tiện nghi cơ bản đầy đủ ^..."
7,1.0,Thuận tiện cho việc đi thăm thú cảnh quan.\r\n...
10,1.0,Nhân viên thân thiện
...,...,...
6994,1.0,"Anh chủ rất là nhiệt tình và vui vẻ, phòng sạc..."
6995,1.0,"Vị trí đẹp, mặc dù chúng tôi là khách duy nhất..."
6996,-1.0,"Không có chỗ đỗ xe, phải gửi xe ở ngoài rất tố..."
6997,1.0,"Gửi quản lý cấp cao của khách sạn, Rất tiếc hì..."


In [6]:
with open('vietnamese_stopwords.txt', encoding="utf8") as f:
    stopwords = []
    for line in f:
        stopwords.append("_".join(line.strip().split()))

In [7]:
import re
from pyvi import ViTokenizer
def preprocessor(text):
    corpus = []
    for i in range(0, len(text)):
        review = re.sub(r"http\S+", "", str(text[i]))
        review = re.sub(r"#\S+", "", review)
        review = re.sub(r"@\S+", "", review)
        review = re.sub('[_]',' ',review)
        review = re.sub('[^a-zA-Z_áàạảãăắằặẵẳâấầẩậẫđíỉìịĩóòỏọõôốồổộỗơớờởợỡéèẹẽẻêếềểệễúùủũụưứừửựữýỳỷỹỵÁÀẢÃẠĂẮẰẲẲẶẴÂẤẦẬẪẨĐÍÌỈỊĨÓÒỎỌÕÔỐỒỔỘỖƠỚỜỞỢỠÉÈẺẸẼÊẾỀỆỂỄÚÙỦŨỤƯỨỪỬỰỮÝỲỶỴỸ]',
                        ' ',review)
        review = ViTokenizer.tokenize(review)
        review = review.lower()
        review = review.split()
        review = [word for word in review if not word in  set(stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [8]:
X = sentiment['text'].values
corpus = preprocessor(X)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(corpus) 
X_data_tfidf =  tfidf_vect.transform(corpus)

from sklearn.model_selection import train_test_split
X = X_data_tfidf
y = sentiment['text_lb'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))

accuracy: 0.8759615384615385


In [10]:
from sklearn import svm
classifier = svm.SVC(probability=True)
classifier.fit(X_train, y_train)
train_predictions = classifier.predict(X_train)
predictions = classifier.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.885576923076923
confusion matrix:
 [[ 84 100]
 [ 19 837]]
classification report:
               precision    recall  f1-score   support

        -1.0       0.82      0.46      0.59       184
         1.0       0.89      0.98      0.93       856

    accuracy                           0.89      1040
   macro avg       0.85      0.72      0.76      1040
weighted avg       0.88      0.89      0.87      1040



In [11]:
X_1 = review_predict.values
corpus1 = preprocessor(X_1)
X_data_tfidf_1 =  tfidf_vect.transform(corpus1)

In [12]:
proba = classifier.predict_proba(X_data_tfidf_1)

In [13]:
df1 = pd.DataFrame(data=proba, columns=['Negative', 'Positive'])

In [14]:
data = pd.concat([data, df1], axis = 1).drop(columns = ['comment'])
data

,index,name,price,number_of_comment,brand,size,weight(g),camera(mp),Negative,Positive
0,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,0.413417,0.586583
1,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,0.500000,0.500000
2,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,0.387752,0.612248
3,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,0.143469,0.856531
4,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,Fujifilm,1076 x 1212 x 673 mm,293.0,NaN,0.299253,0.700747
...,...,...,...,...,...,...,...,...,...,...
1584,196,Máy ảnh Olympus OM-D E-M10 Mark III + Kit 14-4...,14500000,0,Olympus,NaN,NaN,NaN,0.039862,0.960138
1585,197,Máy Ảnh Fujifilm X-Pro 2 + 23mm F2 Black - Hàn...,50480000,0,Fujifilm,Dài 1405mm x Rộng 828mm x Cao 459mm,NaN,NaN,0.039862,0.960138
1586,198,Máy Ảnh Sony DSC-RX100M5A - Hàng Chính Hãng,20990000,0,Sony,1016 x 581 x 410 mm,NaN,NaN,0.039862,0.960138
1587,199,Máy ảnh Olympus E-M1X (Body) ( Hàng Chính hãng),72000000,0,Olympus,NaN,NaN,NaN,0.039862,0.960138


In [15]:
group_data = data.groupby(['index', 'name', 'price', 'number_of_comment', 'size', 'brand', 'weight(g)', 'camera(mp)'], as_index=False, dropna=False).mean()
#data.groupby(['index', 'name', 'price', 'number_of_comment'], as_index=False).mean()
group_data

,index,name,price,number_of_comment,size,brand,weight(g),camera(mp),Negative,Positive
0,0,Máy Ảnh Lấy Liền Fujifilm Instax Mini 11-Hàng ...,1890000,59,1076 x 1212 x 673 mm,Fujifilm,293.0,NaN,0.172505,0.827495
1,1,Máy Ảnh Canon EOS 3000D + Lens EF-S 18 - 55mm ...,7290000,63,NaN,Canon,436.0,NaN,0.282811,0.717189
2,2,Máy Ảnh Fujifilm X-T200 + Lens 15-45mm (24.2MP...,15290000,22,1210 x 837 x 551 mm,Fujifilm,370.0,NaN,0.157547,0.842453
3,3,Máy Ảnh Canon Powershot G7X Mark II – Đen – Hà...,10490000,18,Dài 1097 x Rộng 638 x Dày 357mm,Canon,NaN,NaN,0.248487,0.751513
4,4,Máy Ảnh Fujifilm X-A7 + Lens 15-45mm - Hàng Ch...,11990000,30,119 x 677 x 411 mm,Fujifilm,NaN,NaN,0.292232,0.707768
...,...,...,...,...,...,...,...,...,...,...
196,196,Máy ảnh Olympus OM-D E-M10 Mark III + Kit 14-4...,14500000,0,NaN,Olympus,NaN,NaN,0.039862,0.960138
197,197,Máy Ảnh Fujifilm X-Pro 2 + 23mm F2 Black - Hàn...,50480000,0,Dài 1405mm x Rộng 828mm x Cao 459mm,Fujifilm,NaN,NaN,0.039862,0.960138
198,198,Máy Ảnh Sony DSC-RX100M5A - Hàng Chính Hãng,20990000,0,1016 x 581 x 410 mm,Sony,NaN,NaN,0.039862,0.960138
199,199,Máy ảnh Olympus E-M1X (Body) ( Hàng Chính hãng),72000000,0,NaN,Olympus,NaN,NaN,0.039862,0.960138


In [16]:
rank_data = group_data.drop(columns=['index', 'name', 'brand', 'size'])
rank_data.values

array([['1890000', 59, 293.0, nan, 0.17250469278126926, 0.82749530721873],
       ['7290000', 63, 436.0, nan, 0.28281100623877586,
        0.7171889937612239],
       ['15290000', 22, 370.0, nan, 0.15754691776513716,
        0.8424530822348629],
       ...,
       ['20990000', 0, nan, nan, 0.03986181945723021, 0.9601381805427698],
       ['72000000', 0, nan, nan, 0.03986181945723021, 0.9601381805427698],
       ['4350000', 0, nan, nan, 0.03986181945723021, 0.9601381805427698]],
      dtype=object)

In [17]:
pd.set_option('display.max_rows', rank_data.shape[0]+1)
rank_data[lambda col :rank_data.columns] = rank_data[lambda col :rank_data.columns].replace(r'\D+', np.NaN, regex=True)
rank_data

,price,number_of_comment,weight(g),camera(mp),Negative,Positive
0,1890000,59,293.0,NaN,0.172505,0.827495
1,7290000,63,436.0,NaN,0.282811,0.717189
2,15290000,22,370.0,NaN,0.157547,0.842453
3,10490000,18,NaN,NaN,0.248487,0.751513
4,11990000,30,NaN,NaN,0.292232,0.707768
5,3590000,56,255.0,NaN,0.229838,0.770162
6,23900000,24,NaN,261.0,0.207685,0.792315
7,11990000,193,258.0,243.0,0.319118,0.680882
8,3490000,178,590.0,201.0,0.300641,0.699359
9,16590000,94,387.0,24.0,0.287483,0.712517


In [18]:
from sklearn.impute import KNNImputer
data_imputed = rank_data.values
ix = [i for i in range(data_imputed.shape[1]) if i != 6]
X = data_imputed[:, ix]
imputer = KNNImputer()
imputer.fit(X)
Xtrans = imputer.transform(X)
X.shape

(201, 6)

In [19]:
rank_data = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)

In [20]:
rank_data

,price,number_of_comment,weight(g),camera(mp),Negative,Positive
0,1890000.0,59.0,293.0,90.2,0.172505,0.827495
1,7290000.0,63.0,436.0,190.2,0.282811,0.717189
2,15290000.0,22.0,370.0,174.2,0.157547,0.842453
3,10490000.0,18.0,326.2,242.6,0.248487,0.751513
4,11990000.0,30.0,273.2,234.4,0.292232,0.707768
5,3590000.0,56.0,255.0,90.2,0.229838,0.770162
6,23900000.0,24.0,372.4,261.0,0.207685,0.792315
7,11990000.0,193.0,258.0,243.0,0.319118,0.680882
8,3490000.0,178.0,590.0,201.0,0.300641,0.699359
9,16590000.0,94.0,387.0,24.0,0.287483,0.712517


In [21]:
from sklearn.preprocessing import StandardScaler

Standard_scaler = StandardScaler()
Xtrans = Standard_scaler.fit_transform(Xtrans)

In [22]:
rank_data_scaled = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)

In [23]:
from skcriteria import Data, MIN, MAX
ranking = Data(
    rank_data_scaled,
    [MIN, MAX, MIN, MAX, MIN, MAX],
    cnames = rank_data.columns,
    weights = [1,10,10,1,1,1]
)

In [24]:
from skcriteria.madm import simple
dm = simple.WeightedSum(mnorm = "sum",wnorm = "sum")
dec = dm.decide(ranking)

In [25]:
rank_data['rank'] = dec.rank_
rank_data.sort_values(by=['rank'])

,price,number_of_comment,weight(g),camera(mp),Negative,Positive,rank
51,78000000.0,0.0,845.0,506.0,0.039862,0.960138,1
137,69990000.0,0.0,763.0,457.0,0.039862,0.960138,2
50,38000000.0,2.0,582.0,424.0,0.437488,0.562512,3
156,31990000.0,1.0,501.6,278.0,0.488995,0.511005,4
71,65900000.0,0.0,660.0,314.0,0.039862,0.960138,5
75,62690000.0,0.0,685.0,314.0,0.039862,0.960138,6
95,59990000.0,0.0,890.0,314.0,0.039862,0.960138,7
98,84990000.0,0.0,708.0,314.0,0.039862,0.960138,8
139,69900000.0,0.0,675.0,314.0,0.039862,0.960138,9
146,64990000.0,0.0,699.0,314.0,0.039862,0.960138,10


In [26]:
cols_to_keep = ['index', 'name', 'brand']
group_data = group_data.drop(group_data.columns.difference(cols_to_keep), axis=1)

In [27]:
pd.concat([group_data, rank_data], axis=1, join="inner").to_csv('camera_done.csv')